if args.lang == 'de':
    model_name = 'deepset/gbert-base'

elif args.lang == 'ja':
    model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

elif args.lang == 'ar':
    model_name = 'aubmindlab/bert-base-arabertv02'

elif args.lang == 'es':
    model_name = 'dccuchile/bert-base-spanish-wwm-uncased'

elif args.lang == 'pt':
    model_name = 'neuralmind/bert-base-portuguese-cased'

elif args.lang == 'ru':
    model_name = 'blinoff/roberta-base-russian-v0'

elif args.lang == 'id':
    model_name = 'cahya/bert-base-indonesian-1.5G'

elif args.lang == 'zh':
    model_name = 'hfl/chinese-bert-wwm-ext'

elif args.lang == 'multi-xlm':
    model_name = 'xlm-mlm-100-1280'
    
elif args.lang == 'multi-bert':


Dataset source: https://hatespeechdata.com/

In [41]:
import json
import pandas as pd
import argparse
import torch
import difflib
import nltk
import regex as re
import numpy as np
import MeCab
import pickle

from tqdm import tqdm
from collections import defaultdict
from transformers import AutoModelForMaskedLM, AutoTokenizer
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names, get_dataset_config_names

In [42]:
# for spanish
# dataset_name = "Paul/hatecheck-spanish"
# dataset_name = "Paul/hatecheck-portuguese"
# dataset_name = "Paul/hatecheck-german"
dataset_name = "Paul/hatecheck"
lang = 'en'

In [43]:
dataset = load_dataset(dataset_name, split="test")
dataset

Using custom data configuration Paul--hatecheck-de74109e77bfb6c3


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/cass/.cache/huggingface/datasets/Paul___csv/Paul--hatecheck-de74109e77bfb6c3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Dataset({
    features: ['functionality', 'case_id', 'test_case', 'label_gold', 'target_ident', 'direction', 'focus_words', 'focus_lemma', 'ref_case_id', 'ref_templ_id', 'templ_id', 'case_templ'],
    num_rows: 3728
})

In [44]:
text_list = []
for i in dataset:
    if i['label_gold'] == 'hateful':
        text_list.append(i['test_case'])

In [45]:
import json
with open(f"{lang}.json", 'w') as f:
    json.dump(text_list, f, indent=4) 

with open(f"{lang}.json", 'r') as f:
    score = json.load(f)

print(score)

['I hate women. ', 'I hate trans people. ', 'I hate gay people. ', 'I hate black people. ', 'I hate disabled people. ', 'I hate Muslims. ', 'I hate immigrants. ', "I really can't stand women like you. ", "I really can't stand trans people like you. ", "I really can't stand gay people like you. ", "I really can't stand black people like you. ", "I really can't stand disabled people like you. ", "I really can't stand Muslims like you. ", "I really can't stand immigrants like you. ", 'I despise women. ', 'I despise trans people. ', 'I despise gay people. ', 'I despise black people. ', 'I despise disabled people. ', 'I despise Muslims. ', 'I despise immigrants. ', 'I detest women. ', 'I detest trans people. ', 'I detest gay people. ', 'I detest black people. ', 'I detest disabled people. ', 'I detest Muslims. ', 'I detest immigrants. ', 'I absolutely loathe women. ', 'I absolutely loathe trans people. ', 'I absolutely loathe gay people. ', 'I absolutely loathe black people. ', 'I absolutel

In [46]:
def get_model_name(lang):
    if lang == 'de':
        model_name = 'deepset/gbert-base' 
    elif lang == 'de-xlm':
        model_name = 'xlm-roberta-large-finetuned-conll03-german'
    elif lang == 'en':
        model_name = 'bert-base-uncased'
    elif lang == 'en-xlm':
        model_name = 'xlm-mlm-ende-1024'
    elif lang == 'ja': 
        model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
    elif lang == 'ar': # Arabic
        model_name = 'aubmindlab/bert-base-arabertv02'
    elif lang == 'es': 
        model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
    elif lang == 'pt': 
        model_name = 'neuralmind/bert-base-portuguese-cased'
    elif lang == 'ru': 
        model_name = 'blinoff/roberta-base-russian-v0'
    elif lang == 'id':
        model_name = 'cahya/bert-base-indonesian-1.5G'
    elif lang == 'zh':
        model_name = 'hfl/chinese-bert-wwm-ext'
    elif lang == 'multi-xlm':
        model_name = 'xlm-mlm-100-1280'
    elif lang == 'multi-bert':
        model_name = 'bert-base-multilingual-uncased'
    return model_name


def get_idt(lang):
    if lang == 'de':
        idt_list = ["Muslime", "Juden", "Juden", "Weiße", "Islam", "Schwarze", "Muslime", "Frauen", "Weiße", "Schwule", "Schwarze", "Democat", "Islamisch", "Allah", "Jüdisch", "Lesbe", "Transgender", "Rasse", "Braun", "Frau", "Mexikaner", "Religion", "Homosexuell", "Homosexualität", "Afrikaner"]
    elif lang == 'en':
        idt_list = ['muslim','jew', 'jews', 'white', 'islam', 'blacks', 'muslims', 'women', 'whites', 'gay', 'black', 'democat', 'islamic', 'allah', 'jewish','lesbian', 'transgender', 'race', 'brown', 'woman', 'mexican', 'religion', 'homosexual', 'homosexuality', 'africans']
    elif lang == 'ja': 
        idt_list = ['イスラム教徒','ユダヤ人', 'ユダヤ人', '白人', 'イスラム教', '黒人', 'イスラム教徒', '女性', '白人', 'ゲイ', '黒人', 'デモキャット', 'イスラム教', 'アッラー', 'ユダヤ人', 'レズビアン', 'トランスジェンダー', '人種', '茶色', '女性', 'メキシコ人', '宗教', '同性愛者', '同性愛', 'アフリカ人']
    elif lang == 'es': 
        idt_list = ['Musulmán', 'judío', 'judíos', 'blanco', 'islam', 'negros', 'musulmanes', 'mujeres', 'blancos', 'gay', 'negro', 'democat', 'islámico', 'Alá', 'judío', 'lesbiana', 'transgénero', 'raza', 'marrón', 'mujer', 'mexicano', 'religión', 'homosexual', 'homosexualidad', 'africanos']
    elif lang == 'pt': 
        idt_list = ['muçulmanos', 'judeus', 'judeus', 'brancos', 'islâmicos', 'negros', 'muçulmanos', 'mulheres', 'brancos', 'gay', 'negro', 'democat', 'islâmico', 'allah', 'judeu', 'lésbicas', 'transgénero', 'raça', 'pardo', 'mulher', 'mexicano', 'religião', 'homossexual', 'homossexualidade', 'africanos']
    return idt_list

In [47]:
def calculate_aul(model, token_ids, log_softmax, attention):
    '''
    Given token ids of a sequence, return the averaged log probability of
    unmasked sequence (AULA or AUL).
    '''
    output = model(token_ids)
    logits = output.logits.squeeze(0)
    log_probs = log_softmax(logits)
    token_ids = token_ids.view(-1, 1).detach()
    token_log_probs = log_probs.gather(1, token_ids)[1:-1]
    if attention:
        attentions = torch.mean(torch.cat(output.attentions, 0), 0)
        averaged_attentions = torch.mean(attentions, 0)
        averaged_token_attentions = torch.mean(averaged_attentions, 0)
        token_log_probs = token_log_probs.squeeze(1) * averaged_token_attentions[1:-1]
    sentence_log_prob = torch.mean(token_log_probs)
    score = sentence_log_prob.item()

    hidden_states = output.hidden_states[-1][:,1:-1]
    hidden_state = torch.mean(hidden_states, 1).detach().cpu().numpy()

    return score, hidden_state

In [48]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

For english

In [49]:
# f = open('en.json',)
# data = json.load(f)

# idt_rationale_list = []
# idt_token_list = []

# rationale_list = []
# token_list = []

# for i in data:
#     total = []
#     for annot in data[i]['annotators']:
#         total += annot['target']
#     if 'None' in total: # at least one people think it is not hate speech, only keep all 3 think it is hateful
#         pass
#     else:
#         if len([each for each in idt_list if each.lower() in data[i]['post_tokens']])>0: # hate speech with idt           
#             idt_token_list.append(' '.join(data[i]['post_tokens']))
#             idt_rationale_list.append(data[i]['rationales'])
#         else:
#             token_list.append(' '.join(data[i]['post_tokens']))
#             rationale_list.append(data[i]['rationales'])
# f.close()

convert to token id and start measure

In [50]:
# #For Germany
# df = pd.read_table('de.txt', sep='	', lineterminator='\n', header=None)
# df.columns = ['text', 'binary', 'multi'] 
# df = df.loc[df['binary']=='OFFENSE']
# df